# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 17 2023 11:26AM,255212,10,Yusen-9042395,Yusen – 3D Matrix,Released
1,Jan 17 2023 11:18AM,255211,19,ACG-2102492472,ACG North America LLC,Released
2,Jan 17 2023 11:16AM,255210,19,ACG-2102492468,ACG North America LLC,Released
3,Jan 17 2023 11:10AM,255209,19,TR-RG-01182023-WM225,"GUSA Granules USA, Inc.",Released
4,Jan 17 2023 11:06AM,255208,10,CTF0011601,"Citieffe, Inc.",Released
5,Jan 17 2023 11:05AM,255098,15,9037907,"Brookfield Pharmaceuticals, LLC",Released
6,Jan 17 2023 11:05AM,255098,15,9037908,"Brookfield Pharmaceuticals, LLC",Released
7,Jan 17 2023 11:05AM,255098,15,9037909,"Brookfield Pharmaceuticals, LLC",Released
8,Jan 17 2023 11:05AM,255098,15,9037910,"Brookfield Pharmaceuticals, LLC",Released
9,Jan 17 2023 11:05AM,255098,15,9037911,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,255208,Released,1
30,255209,Released,1
31,255210,Released,1
32,255211,Released,1
33,255212,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
255208,NaN,1.0
255209,NaN,1.0
255210,NaN,1.0
255211,NaN,1.0
255212,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
255097,41.0,3.0
255098,0.0,22.0
255101,0.0,1.0
255157,18.0,82.0
255162,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
255097,41,3
255098,0,22
255101,0,1
255157,18,82
255162,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,255097,41,3
1,255098,0,22
2,255101,0,1
3,255157,18,82
4,255162,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,255097,41,3
1,255098,,22
2,255101,,1
3,255157,18,82
4,255162,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix
1,Jan 17 2023 11:18AM,255211,19,ACG North America LLC
2,Jan 17 2023 11:16AM,255210,19,ACG North America LLC
3,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc."
4,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc."
5,Jan 17 2023 11:05AM,255098,15,"Brookfield Pharmaceuticals, LLC"
27,Jan 17 2023 11:01AM,255207,19,"AdvaGen Pharma, Ltd"
29,Jan 17 2023 11:00AM,255101,15,"Person & Covey, Inc."
30,Jan 17 2023 10:57AM,255162,15,"Alliance Pharma, Inc."
31,Jan 17 2023 10:57AM,255206,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,,1
1,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,,1
2,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,,1
3,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",,1
4,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",,1
5,Jan 17 2023 11:05AM,255098,15,"Brookfield Pharmaceuticals, LLC",,22
6,Jan 17 2023 11:01AM,255207,19,"AdvaGen Pharma, Ltd",,2
7,Jan 17 2023 11:00AM,255101,15,"Person & Covey, Inc.",,1
8,Jan 17 2023 10:57AM,255162,15,"Alliance Pharma, Inc.",,1
9,Jan 17 2023 10:57AM,255206,19,"AdvaGen Pharma, Ltd",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,1,
1,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,1,
2,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,1,
3,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",1,
4,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",1,
5,Jan 17 2023 11:05AM,255098,15,"Brookfield Pharmaceuticals, LLC",22,
6,Jan 17 2023 11:01AM,255207,19,"AdvaGen Pharma, Ltd",2,
7,Jan 17 2023 11:00AM,255101,15,"Person & Covey, Inc.",1,
8,Jan 17 2023 10:57AM,255162,15,"Alliance Pharma, Inc.",1,
9,Jan 17 2023 10:57AM,255206,19,"AdvaGen Pharma, Ltd",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,1,
1,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,1,
2,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,1,
3,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",1,
4,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,1,NaN
1,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,1,NaN
2,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,1,NaN
3,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",1,NaN
4,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 17 2023 11:26AM,255212,10,Yusen – 3D Matrix,1,0.0
1,Jan 17 2023 11:18AM,255211,19,ACG North America LLC,1,0.0
2,Jan 17 2023 11:16AM,255210,19,ACG North America LLC,1,0.0
3,Jan 17 2023 11:10AM,255209,19,"GUSA Granules USA, Inc.",1,0.0
4,Jan 17 2023 11:06AM,255208,10,"Citieffe, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3317446,120,58.0
12,765591,3,0.0
15,1020518,106,18.0
19,2041622,29,0.0
20,255198,1,0.0
22,255187,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3317446,120,58.0
1,12,765591,3,0.0
2,15,1020518,106,18.0
3,19,2041622,29,0.0
4,20,255198,1,0.0
5,22,255187,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,120,58.0
1,12,3,0.0
2,15,106,18.0
3,19,29,0.0
4,20,1,0.0
5,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,120.0
1,12,Released,3.0
2,15,Released,106.0
3,19,Released,29.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,22
Status,,,,,,
Executing,58.0,0.0,18.0,0.0,0.0,0.0
Released,120.0,3.0,106.0,29.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,22
0,Executing,58.0,0.0,18.0,0.0,0.0,0.0
1,Released,120.0,3.0,106.0,29.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,22
0,Executing,58.0,0.0,18.0,0.0,0.0,0.0
1,Released,120.0,3.0,106.0,29.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()